In [1]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import sparse
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import zscore
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math
from matplotlib.pyplot import *

In [2]:
class Reservoir(object):
    """
    Build a reservoir and evaluate internal states
    
    Parameters:
        n_internal_units = processing units in the reservoir
        spectral_radius = largest eigenvalue of the reservoir matrix of connection weights
        leak = amount of leakage in the reservoir state update (optional)
        connectivity = percentage of nonzero connection weights (unused in circle reservoir)
        input_scaling = scaling of the input connection weights
        noise_level = deviation of the Gaussian noise injected in the state update
        circle = generate determinisitc reservoir with circle topology
    """
    
    def __init__(self, n_internal_units=100, spectral_radius=0.99, leak=None,
                 connectivity=0.3, input_scaling=0.2, noise_level=0.01, circle=False):
        
        # Initialize attributes
        self._n_internal_units = n_internal_units
        self._input_scaling = input_scaling
        self._noise_level = noise_level
        self._leak = leak

        # Input weights depend on input size: they are set when data is provided
        self._input_weights = None

        # Generate internal weights
        if circle:
            self._internal_weights = self._initialize_internal_weights_Circ(
                    n_internal_units,
                    spectral_radius)
        else:
            self._internal_weights = self._initialize_internal_weights(
                n_internal_units,
                connectivity,
                spectral_radius)


    def _initialize_internal_weights_Circ(self, n_internal_units, spectral_radius):
        
        internal_weights = np.zeros((n_internal_units, n_internal_units))
        internal_weights[0,-1] = spectral_radius
        for i in range(n_internal_units-1):
            internal_weights[i+1,i] = spectral_radius
                
        return internal_weights
    
    
    def _initialize_internal_weights(self, n_internal_units,
                                     connectivity, spectral_radius):

        # Generate sparse, uniformly distributed weights.
        internal_weights = sparse.rand(n_internal_units,
                                       n_internal_units,
                                       density=connectivity).todense()

        # Ensure that the nonzero values are uniformly distributed in [-0.5, 0.5]
        internal_weights[np.where(internal_weights > 0)] -= 0.5
        
        # Adjust the spectral radius.
        E, _ = np.linalg.eig(internal_weights)
        e_max = np.max(np.abs(E))
        internal_weights /= np.abs(e_max)/spectral_radius       

        return internal_weights


    def _compute_state_matrix(self, X, n_drop=0):
        N, T, _ = X.shape
        previous_state = np.zeros((N, self._n_internal_units), dtype=float)

        # Storage
        state_matrix = np.empty((N, T - n_drop, self._n_internal_units), dtype=float)
        for t in range(T):
            current_input = X[:, t, :]

            # Calculate state
            state_before_tanh = self._internal_weights.dot(previous_state.T) + self._input_weights.dot(current_input.T)

            # Add noise
            state_before_tanh += np.random.rand(self._n_internal_units, N)*self._noise_level

            # Apply nonlinearity and leakage (optional)
            if self._leak is None:
                previous_state = np.tanh(state_before_tanh).T
            else:
                previous_state = (1.0 - self._leak)*previous_state + np.tanh(state_before_tanh).T

            # Store everything after the dropout period
            if (t > n_drop - 1):
                state_matrix[:, t - n_drop, :] = previous_state

        return state_matrix


    def get_states(self, X, n_drop=0, bidir=True):
        N, T, V = X.shape
        if self._input_weights is None:
            self._input_weights = (2.0*np.random.binomial(1, 0.5 , [self._n_internal_units, V]) - 1.0)*self._input_scaling

        # compute sequence of reservoir states
        states = self._compute_state_matrix(X, n_drop)
    
        # reservoir states on time reversed input
        if bidir is True:
            X_r = X[:, ::-1, :]
            states_r = self._compute_state_matrix(X_r, n_drop)
            states = np.concatenate((states, states_r), axis=2)

        return states
    
    def getReservoirEmbedding(self, X,pca, ridge_embedding,  n_drop=5, bidir=True, test = False):

        res_states = self.get_states(X, n_drop=5, bidir=True)


        N_samples = res_states.shape[0]
        res_states = res_states.reshape(-1, res_states.shape[2])                   
        # ..transform..
        if test:
            red_states = pca.transform(res_states)
        else:
            red_states = pca.fit_transform(res_states)          
        # ..and put back in tensor form
        red_states = red_states.reshape(N_samples,-1,red_states.shape[1])  

        coeff_tr = []
        biases_tr = []   

        for i in range(X.shape[0]):
            ridge_embedding.fit(red_states[i, 0:-1, :], red_states[i, 1:, :])
            coeff_tr.append(ridge_embedding.coef_.ravel())
            biases_tr.append(ridge_embedding.intercept_.ravel())
        #print(np.array(coeff_tr).shape,np.array(biases_tr).shape)
        input_repr = np.concatenate((np.vstack(coeff_tr), np.vstack(biases_tr)), axis=1)
        return input_repr

In [3]:
data = pd.read_csv("Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv")

In [3]:
data = data.sample(frac=0.5, replace=True, random_state=1)
data.head()

# plot some of it
#figure(10).clear()
#plot(data[0:1000])
#title('A sample of data')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
128037,53,17,15/02/2018 11:18:07,2261,1,1,49,164,49,49,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign
491755,3389,6,15/02/2018 01:57:33,2837721,9,7,1144,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
470924,3389,6,15/02/2018 09:21:14,1645626,8,7,1148,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
791624,3389,6,15/02/2018 02:46:24,2486564,8,8,1144,1874,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
491263,53,17,15/02/2018 12:34:09,1758,1,1,30,207,30,30,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign


In [4]:
# features = ['Flow Duration', 'Tot Fwd Pkts',
#        'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
#        'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
#        'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
#        'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
#        'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
#        'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
#        'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
#        'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
#        'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
#        'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
#        'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
#        'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
#        'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
#        'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg',
#        'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
#        'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
#        'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
#        'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts',
#        'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
#        'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']

In [5]:
features = ['Fwd Seg Size Min', 'Init Fwd Win Byts', 'Bwd Pkt Len Max', 'Bwd IAT Mean', 'Fwd IAT Min', 'Flow IAT Min', 'Flow IAT Mean', 'Bwd IAT Max', 'Fwd IAT Mean', 'Fwd IAT Tot']

In [6]:
def targetify(s):
    if s == 'Benign':
        return 0
    else:
        return 1

In [7]:
num_col = data.shape[1]
num_row = data.shape[0]

X_data = data[features]
X_data[features] = X_data[features].apply(pd.to_numeric, errors='coerce', axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_data.values)
X = pd.DataFrame(np.nan_to_num(x_scaled))
if len(X.shape) < 3:
    X = np.atleast_3d(X)
y = data['Label'].apply(targetify)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [8]:
X.shape, y.shape

((524288, 10, 1), (524288,))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75)

# Working with 0.01% of the data for now because my laptop sucks
#X_train2 = X_train.sample(frac=0.0005, replace=True, random_state=1)
#y_train2 = y_train.sample(frac=0.0005, replace=True, random_state=1)
#X_test2 = X_test.sample(frac=0.0005, replace=True, random_state=1)
#y_test2 = y_test.sample(frac=0.0005, replace=True, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(131072, 10, 1) (131072,)
(393216, 10, 1) (393216,)


In [10]:
pca = PCA() #n_components gives number of components to keep for linear dimensionality reduction
ridge_embedding = Ridge(alpha=10, fit_intercept=True)
readout = Ridge(alpha=5)

# Changing # of internal units
Using 50% of the dataset, 10 features, 5 internal units

In [11]:
res = Reservoir(n_internal_units=5, spectral_radius=0.6, leak=0.6,
                 connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)

In [12]:
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)

In [13]:
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)

In [14]:
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
print(pred_class)
predictions = [int(round(x)) for x in pred_class]
print(np.count_nonzero(predictions))

[ 0.05476859 -0.04400997 -0.05940531 ...  0.11196675  0.1573708
 -0.05228745]
0


In [15]:
true_class = list(y_test)

In [19]:
df = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
df = df.sort_values('pred_class', ascending=False)

In [20]:
df.to_csv('results_0.5_10_5.csv')

In [21]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)

#accuracy_score(true_class, pred_class, normalize=False)

0.9499969482421875

Using 50% of the dataset, 10 features, 10 internal units

In [30]:
res = Reservoir(n_internal_units=10, spectral_radius=0.6, leak=0.6,
                 connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)

In [31]:
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)
input_repr

array([[ 2.25659963e-04, -1.42649349e-04,  1.73539492e-04, ...,
        -1.37841530e-04,  1.48743551e-03, -2.49790989e-03],
       [ 1.07158278e-05,  4.81017286e-05,  2.99228292e-05, ...,
        -3.08480095e-03, -1.01975257e-03, -2.14090261e-03],
       [-2.37777903e-05,  2.67257324e-05,  4.75751928e-05, ...,
         1.71379704e-03,  3.51352758e-04, -3.19495580e-04],
       ...,
       [ 1.59754603e-04, -2.55475548e-05,  4.11445377e-04, ...,
        -2.68032602e-03,  6.50308939e-04, -2.73899293e-03],
       [ 1.28617368e-03, -8.83345639e-04,  3.11659642e-04, ...,
         9.85300412e-04, -6.63025907e-05, -8.00904197e-04],
       [-3.08221620e-06, -3.76070711e-05,  3.77970104e-05, ...,
         1.16210023e-03, -4.81899655e-04,  1.58700907e-03]])

In [32]:
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)
input_repr_te

array([[ 1.88748946e-04, -1.77526562e-04, -1.66993238e-05, ...,
        -7.72788784e-05,  6.50809076e-04, -4.31129275e-04],
       [ 6.77046164e-06,  5.63176869e-06,  1.86921682e-05, ...,
         4.98619548e-04, -1.49953954e-03,  1.95601476e-04],
       [ 5.49031536e-06,  7.41776605e-06,  7.89243959e-06, ...,
        -2.50221153e-03,  8.49649748e-04, -7.21088086e-04],
       ...,
       [ 2.05569091e-05, -1.74666605e-06,  1.99202573e-04, ...,
         6.20133466e-04,  1.74525017e-03,  6.02902579e-04],
       [ 1.70269486e-05, -3.98875124e-07,  2.70031713e-04, ...,
         1.76196936e-03,  2.69422220e-04,  6.47542951e-04],
       [-1.95909665e-05,  4.59862370e-06,  3.95693855e-05, ...,
         2.03730079e-04,  3.00013049e-03,  3.09561711e-03]])

In [33]:
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
print(pred_class)
predictions = [int(round(x)) for x in pred_class]
print(np.count_nonzero(predictions))

[ 0.07295991 -0.01904003 -0.00768316 ...  0.14255407  0.1379468
 -0.06404667]
0


In [34]:
true_class = list(y_test)

In [35]:
df = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
df = df.sort_values('pred_class', ascending=False)

In [36]:
df.to_csv('results_0.5_10_10.csv')

In [37]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)

0.9499969482421875

Using 50% of the dataset, 10 features, 15 internal units

In [22]:
res = Reservoir(n_internal_units=15, spectral_radius=0.6, leak=0.6,
                 connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)

In [23]:
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)
input_repr

array([[ 2.90381479e-04, -7.45808866e-04,  2.59981747e-04, ...,
        -8.89440435e-04, -5.21813538e-04,  3.78180570e-04],
       [-7.85279480e-05, -1.02692144e-04,  1.37144577e-04, ...,
         1.87454599e-03,  1.01951171e-04,  2.43643233e-04],
       [-1.60544566e-05,  6.38639421e-07,  9.56687334e-06, ...,
         2.07238516e-03, -2.11093355e-03, -2.50821483e-04],
       ...,
       [ 5.42912573e-04, -1.66669625e-03,  2.25633277e-04, ...,
         1.31629135e-03, -2.33994652e-03, -1.86783770e-03],
       [ 1.19945789e-03, -2.94509942e-03,  5.16684703e-04, ...,
        -6.89148991e-04, -1.97991466e-03, -7.73689522e-04],
       [-4.08305470e-05, -1.17015565e-04,  1.15919114e-04, ...,
        -3.71325027e-03,  1.03504254e-03,  1.05640727e-03]])

In [24]:
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)
input_repr_te

array([[ 6.33405385e-05, -4.45267094e-04,  6.43532341e-05, ...,
         2.91614745e-03, -1.11913948e-03,  2.31431646e-03],
       [-1.37389415e-05, -2.14771416e-05,  8.60692494e-06, ...,
         5.39520825e-04,  2.57789446e-03, -8.75540229e-04],
       [-1.87057909e-05, -8.21976622e-07,  1.30422329e-05, ...,
         2.37446940e-04, -4.18232521e-05, -5.92005871e-04],
       ...,
       [ 5.54069029e-04, -1.91219159e-03,  1.73656897e-04, ...,
         2.71629150e-03,  1.73176226e-03, -5.74150088e-04],
       [ 4.38478037e-04, -1.38687488e-03,  1.57951166e-04, ...,
         8.04158029e-04,  1.30543889e-05,  9.77362003e-04],
       [-6.56146204e-06, -2.39881917e-05,  2.28435470e-05, ...,
        -6.93058440e-04,  1.33450285e-03, -6.39709706e-04]])

In [25]:
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
print(pred_class)
predictions = [int(round(x)) for x in pred_class]
print(np.count_nonzero(predictions))

[ 0.06262571 -0.01896104 -0.05851126 ...  0.14705568  0.13050566
 -0.06996091]
0


In [26]:
true_class = list(y_test)

In [27]:
df = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
df = df.sort_values('pred_class', ascending=False)

In [28]:
df.to_csv('results_0.5_10_15.csv')

In [29]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)

0.9499969482421875

Using 50% of the dataset, 10 features, 25 internal units

In [38]:
res = Reservoir(n_internal_units=25, spectral_radius=0.6, leak=0.6,
                 connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)

In [39]:
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)
input_repr

array([[ 0.00217089, -0.00194204, -0.0007908 , ...,  0.00206354,
         0.00073781, -0.00178357],
       [ 0.00052123, -0.00046758, -0.00125267, ..., -0.00044201,
         0.0013192 , -0.00123976],
       [ 0.00013131, -0.00023071, -0.00023145, ...,  0.00189205,
        -0.00051831,  0.00085522],
       ...,
       [ 0.0044493 , -0.00460986, -0.00099087, ...,  0.00020291,
        -0.00310414, -0.000312  ],
       [ 0.00933021, -0.00862896, -0.00235723, ...,  0.00169616,
        -0.00066229, -0.0004821 ],
       [ 0.00049286, -0.00050942, -0.00103795, ..., -0.00034316,
         0.00061552, -0.00177724]])

In [40]:
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)
input_repr_te

MemoryError: Unable to allocate array with shape (393216, 2500) and data type float64

In [ ]:
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
print(pred_class)
predictions = [int(round(x)) for x in pred_class]
print(np.count_nonzero(predictions))

In [ ]:
true_class = list(y_test)

In [ ]:
df = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
df = df.sort_values('pred_class', ascending=False)

In [ ]:
df.to_csv('results_0.5_10_10.csv')

In [ ]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)

# Changing the % of dataset

# Using 75% of the dataset, 10 features, 5 internal units

In [3]:
data = data.sample(frac=0.75, replace=True, random_state=1)
data.head()

# plot some of it
#figure(10).clear()
#plot(data[0:1000])
#title('A sample of data')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
128037,53,17,15/02/2018 11:18:07,2261,1,1,49,164,49,49,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign
491755,3389,6,15/02/2018 01:57:33,2837721,9,7,1144,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
470924,3389,6,15/02/2018 09:21:14,1645626,8,7,1148,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
791624,3389,6,15/02/2018 02:46:24,2486564,8,8,1144,1874,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
491263,53,17,15/02/2018 12:34:09,1758,1,1,30,207,30,30,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign


In [4]:
# features = ['Flow Duration', 'Tot Fwd Pkts',
#        'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
#        'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
#        'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
#        'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
#        'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
#        'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
#        'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
#        'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
#        'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
#        'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
#        'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
#        'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
#        'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
#        'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg',
#        'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
#        'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
#        'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
#        'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts',
#        'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
#        'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']

In [5]:
features = ['Fwd Seg Size Min', 'Init Fwd Win Byts', 'Bwd Pkt Len Max', 'Bwd IAT Mean', 'Fwd IAT Min', 'Flow IAT Min', 'Flow IAT Mean', 'Bwd IAT Max', 'Fwd IAT Mean', 'Fwd IAT Tot']

In [6]:
def targetify(s):
    if s == 'Benign':
        return 0
    else:
        return 1

In [7]:
num_col = data.shape[1]
num_row = data.shape[0]

X_data = data[features]
X_data[features] = X_data[features].apply(pd.to_numeric, errors='coerce', axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_data.values)
X = pd.DataFrame(np.nan_to_num(x_scaled))
if len(X.shape) < 3:
    X = np.atleast_3d(X)
y = data['Label'].apply(targetify)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [8]:
X.shape, y.shape

((524288, 10, 1), (524288,))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75)

# Working with 0.01% of the data for now because my laptop sucks
#X_train2 = X_train.sample(frac=0.0005, replace=True, random_state=1)
#y_train2 = y_train.sample(frac=0.0005, replace=True, random_state=1)
#X_test2 = X_test.sample(frac=0.0005, replace=True, random_state=1)
#y_test2 = y_test.sample(frac=0.0005, replace=True, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(131072, 10, 1) (131072,)
(393216, 10, 1) (393216,)


In [10]:
pca = PCA() #n_components gives number of components to keep for linear dimensionality reduction
ridge_embedding = Ridge(alpha=10, fit_intercept=True)
readout = Ridge(alpha=5)

# Using 50% of the dataset, 10 features, 5 internal units

In [11]:
res = Reservoir(n_internal_units=5, spectral_radius=0.6, leak=0.6,
                 connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)

In [12]:
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)

In [13]:
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)

In [14]:
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
print(pred_class)
predictions = [int(round(x)) for x in pred_class]
print(np.count_nonzero(predictions))

[ 0.05476859 -0.04400997 -0.05940531 ...  0.11196675  0.1573708
 -0.05228745]
0


In [15]:
true_class = list(y_test)

In [19]:
df = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
df = df.sort_values('pred_class', ascending=False)

In [20]:
df.to_csv('results_0.5_10_5.csv')

In [21]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)

#accuracy_score(true_class, pred_class, normalize=False)

0.9499969482421875

# Using 50% of the dataset, 10 features, 10 internal units

In [30]:
res = Reservoir(n_internal_units=10, spectral_radius=0.6, leak=0.6,
                 connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)

In [31]:
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)
input_repr

array([[ 2.25659963e-04, -1.42649349e-04,  1.73539492e-04, ...,
        -1.37841530e-04,  1.48743551e-03, -2.49790989e-03],
       [ 1.07158278e-05,  4.81017286e-05,  2.99228292e-05, ...,
        -3.08480095e-03, -1.01975257e-03, -2.14090261e-03],
       [-2.37777903e-05,  2.67257324e-05,  4.75751928e-05, ...,
         1.71379704e-03,  3.51352758e-04, -3.19495580e-04],
       ...,
       [ 1.59754603e-04, -2.55475548e-05,  4.11445377e-04, ...,
        -2.68032602e-03,  6.50308939e-04, -2.73899293e-03],
       [ 1.28617368e-03, -8.83345639e-04,  3.11659642e-04, ...,
         9.85300412e-04, -6.63025907e-05, -8.00904197e-04],
       [-3.08221620e-06, -3.76070711e-05,  3.77970104e-05, ...,
         1.16210023e-03, -4.81899655e-04,  1.58700907e-03]])

In [32]:
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)
input_repr_te

array([[ 1.88748946e-04, -1.77526562e-04, -1.66993238e-05, ...,
        -7.72788784e-05,  6.50809076e-04, -4.31129275e-04],
       [ 6.77046164e-06,  5.63176869e-06,  1.86921682e-05, ...,
         4.98619548e-04, -1.49953954e-03,  1.95601476e-04],
       [ 5.49031536e-06,  7.41776605e-06,  7.89243959e-06, ...,
        -2.50221153e-03,  8.49649748e-04, -7.21088086e-04],
       ...,
       [ 2.05569091e-05, -1.74666605e-06,  1.99202573e-04, ...,
         6.20133466e-04,  1.74525017e-03,  6.02902579e-04],
       [ 1.70269486e-05, -3.98875124e-07,  2.70031713e-04, ...,
         1.76196936e-03,  2.69422220e-04,  6.47542951e-04],
       [-1.95909665e-05,  4.59862370e-06,  3.95693855e-05, ...,
         2.03730079e-04,  3.00013049e-03,  3.09561711e-03]])

In [33]:
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
print(pred_class)
predictions = [int(round(x)) for x in pred_class]
print(np.count_nonzero(predictions))

[ 0.07295991 -0.01904003 -0.00768316 ...  0.14255407  0.1379468
 -0.06404667]
0


In [34]:
true_class = list(y_test)

In [35]:
df = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
df = df.sort_values('pred_class', ascending=False)

In [36]:
df.to_csv('results_0.5_10_10.csv')

In [37]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)

0.9499969482421875

# Using 50% of the dataset, 10 features, 15 internal units

In [22]:
res = Reservoir(n_internal_units=15, spectral_radius=0.6, leak=0.6,
                 connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)

In [23]:
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)
input_repr

array([[ 2.90381479e-04, -7.45808866e-04,  2.59981747e-04, ...,
        -8.89440435e-04, -5.21813538e-04,  3.78180570e-04],
       [-7.85279480e-05, -1.02692144e-04,  1.37144577e-04, ...,
         1.87454599e-03,  1.01951171e-04,  2.43643233e-04],
       [-1.60544566e-05,  6.38639421e-07,  9.56687334e-06, ...,
         2.07238516e-03, -2.11093355e-03, -2.50821483e-04],
       ...,
       [ 5.42912573e-04, -1.66669625e-03,  2.25633277e-04, ...,
         1.31629135e-03, -2.33994652e-03, -1.86783770e-03],
       [ 1.19945789e-03, -2.94509942e-03,  5.16684703e-04, ...,
        -6.89148991e-04, -1.97991466e-03, -7.73689522e-04],
       [-4.08305470e-05, -1.17015565e-04,  1.15919114e-04, ...,
        -3.71325027e-03,  1.03504254e-03,  1.05640727e-03]])

In [24]:
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)
input_repr_te

array([[ 6.33405385e-05, -4.45267094e-04,  6.43532341e-05, ...,
         2.91614745e-03, -1.11913948e-03,  2.31431646e-03],
       [-1.37389415e-05, -2.14771416e-05,  8.60692494e-06, ...,
         5.39520825e-04,  2.57789446e-03, -8.75540229e-04],
       [-1.87057909e-05, -8.21976622e-07,  1.30422329e-05, ...,
         2.37446940e-04, -4.18232521e-05, -5.92005871e-04],
       ...,
       [ 5.54069029e-04, -1.91219159e-03,  1.73656897e-04, ...,
         2.71629150e-03,  1.73176226e-03, -5.74150088e-04],
       [ 4.38478037e-04, -1.38687488e-03,  1.57951166e-04, ...,
         8.04158029e-04,  1.30543889e-05,  9.77362003e-04],
       [-6.56146204e-06, -2.39881917e-05,  2.28435470e-05, ...,
        -6.93058440e-04,  1.33450285e-03, -6.39709706e-04]])

In [25]:
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
print(pred_class)
predictions = [int(round(x)) for x in pred_class]
print(np.count_nonzero(predictions))

[ 0.06262571 -0.01896104 -0.05851126 ...  0.14705568  0.13050566
 -0.06996091]
0


In [26]:
true_class = list(y_test)

In [27]:
df = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
df = df.sort_values('pred_class', ascending=False)

In [28]:
df.to_csv('results_0.5_10_15.csv')

In [29]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)

0.9499969482421875

# Using 50% of the dataset, 10 features, 25 internal units

In [38]:
res = Reservoir(n_internal_units=25, spectral_radius=0.6, leak=0.6,
                 connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)

In [39]:
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)
input_repr

array([[ 0.00217089, -0.00194204, -0.0007908 , ...,  0.00206354,
         0.00073781, -0.00178357],
       [ 0.00052123, -0.00046758, -0.00125267, ..., -0.00044201,
         0.0013192 , -0.00123976],
       [ 0.00013131, -0.00023071, -0.00023145, ...,  0.00189205,
        -0.00051831,  0.00085522],
       ...,
       [ 0.0044493 , -0.00460986, -0.00099087, ...,  0.00020291,
        -0.00310414, -0.000312  ],
       [ 0.00933021, -0.00862896, -0.00235723, ...,  0.00169616,
        -0.00066229, -0.0004821 ],
       [ 0.00049286, -0.00050942, -0.00103795, ..., -0.00034316,
         0.00061552, -0.00177724]])

In [ ]:
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)
input_repr_te

In [ ]:
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
print(pred_class)
predictions = [int(round(x)) for x in pred_class]
print(np.count_nonzero(predictions))

In [ ]:
true_class = list(y_test)

In [ ]:
df = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
df = df.sort_values('pred_class', ascending=False)

In [ ]:
df.to_csv('results_0.5_10_10.csv')

In [ ]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)